In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import csv
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score

### Data Setup

In [12]:
all_continents = ['afr', 'am', 'asia', 'eur', 'me']
all_data = {}

feature_names = ["warstds", "ager", "agexp", "anoc", "army85", "autch98", "auto4",
"autonomy", "avgnabo", "centpol3", "coldwar", "decade1", "decade2",
"decade3", "decade4", "dem", "dem4", "demch98", "dlang", "drel",
"durable", "ef", "ef2", "ehet", "elfo", "elfo2", "etdo4590",
"expgdp", "exrec", "fedpol3", "fuelexp", "gdpgrowth", "geo1", "geo2",
"geo34", "geo57", "geo69", "geo8", "illiteracy", "incumb", "infant",
"inst", "inst3", "life", "lmtnest", "ln_gdpen", "lpopns", "major", "manuexp", "milper",
"mirps0", "mirps1", "mirps2", "mirps3", "nat_war", "ncontig",
"nmgdp", "nmdp4_alt", "numlang", "nwstate", "oil", "p4mchg",
"parcomp", "parreg", "part", "partfree", "plural", "plurrel",
"pol4", "pol4m", "pol4sq", "polch98", "polcomp", "popdense",
"presi", "pri", "proxregc", "ptime", "reg", "regd4_alt", "relfrac", "seceduc",
"second", "semipol3", "sip2", "sxpnew", "sxpsq", "tnatwar", "trade",
"warhist", "xconst"]

for cont in all_continents:
    for ver in ['tr', 'test']:
        data = []
        names = []
        with open('geo_data/{0}_{1}.csv'.format(cont, ver), newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',', quotechar='|')
            for i, row in enumerate(reader):
                if i == 0:
                    names = row
                else:
                    data.append(row)
        data = np.array(data)
        feature_indices = [names.index(feat) for feat in feature_names[1:]]
        features = data[:, feature_indices]
        features = features.astype(float)
        labels = data[:, names.index('warstds')]
        labels = [int(float(label)) for label in labels]
        all_data['{0}_{1}'.format(cont, ver)] = [features, labels]
        

In [67]:
# hyperparameters
rdm_depth = 5
reg = 100

### Baseline: Train on all continents, disaggregated test results on each continent

In [74]:
# train on all data
all_train_features = []
all_train_labels = []
for key in all_data.keys():
    if 'tr' in key:
        all_train_features.append(all_data[key][0])
        all_train_labels.append(all_data[key][1])
all_train_features = np.concatenate(all_train_features, axis=0)
all_train_labels = np.concatenate(all_train_labels, axis=0)

log_reg = LogisticRegression(C=reg, class_weight='balanced').fit(features, labels)
log_reg_score = log_reg.score(features, labels)
rdm_for = RandomForestClassifier(max_depth=rdm_depth, class_weight='balanced').fit(features, labels)
rdm_for_score = rdm_for.score(features, labels)
print("Training Scores for Logistic Regression: {0}, Random Forest: {1}\n".format(log_reg_score, rdm_for_score))
baseline_accs = []

for key in all_data.keys():
    if 'test' in key:
        test_features = all_data[key][0]
        test_labels = all_data[key][1]
        log_reg_score = log_reg.score(test_features, test_labels)
        rdm_for_score = rdm_for.score(test_features, test_labels)
        log_reg_f1score = f1_score(test_labels, log_reg.predict(test_features))
        rdm_for_f1score = f1_score(test_labels, rdm_for.predict(test_features))
        log_reg_auc = roc_auc_score(test_labels, log_reg.predict_proba(test_features)[:, 1])
        rdm_for_auc = roc_auc_score(test_labels, rdm_for.predict_proba(test_features)[:, 1])
        baseline_accs.append([log_reg_auc, rdm_for_auc])
        print("Test on {0}, LR: {1}, RF: {2}".format(key.split('_')[0], round(log_reg_score, 5), round(rdm_for_score, 5)))
        print("F1 for LR: {0}, RF: {1}".format(round(log_reg_f1score, 4), round(rdm_for_f1score, 4)))
        print("AUC for LR: {0}, RF: {1}\n".format(round(log_reg_auc, 4), round(rdm_for_auc, 4)))
        

Training Scores for Logistic Regression: 1.0, Random Forest: 0.9906976744186047

Test on eur, LR: 0.97603, RF: 0.99782
F1 for LR: 0.0, RF: 0.0
AUC for LR: 0.9541, RF: 0.9782

Test on asia, LR: 0.89896, RF: 0.98446
F1 for LR: 0.0488, RF: 0.0
AUC for LR: 0.6338, RF: 0.6737

Test on afr, LR: 0.94854, RF: 0.97599
F1 for LR: 0.0, RF: 0.0
AUC for LR: 0.4365, RF: 0.7036

Test on am, LR: 0.94037, RF: 0.98624
F1 for LR: 0.0, RF: 0.0
AUC for LR: 0.5729, RF: 0.7671

Test on me, LR: 1.0, RF: 0.9907
F1 for LR: 1.0, RF: 0.6667
AUC for LR: 1.0, RF: 1.0



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Exp: Train on all continents except for x, and then test on x

In [75]:
oos_accs = []
for key in all_data.keys():
    if 'test' in key:
        cont = key.split('_')[0]
        
        train_continents = all_continents.copy()
        train_continents.remove(cont)
        all_train_features = []
        all_train_labels = []
        for t_cont in train_continents:
            all_train_features.append(all_data['{0}_tr'.format(t_cont)][0])
            all_train_labels.append(all_data['{0}_tr'.format(t_cont)][1])
        all_train_features = np.concatenate(all_train_features, axis=0)
        all_train_labels = np.concatenate(all_train_labels, axis=0)

        log_reg = LogisticRegression(C=reg, class_weight='balanced').fit(features, labels)
        rdm_for = RandomForestClassifier(max_depth=rdm_depth, class_weight='balanced').fit(features, labels)
        
        test_features = all_data[key][0]
        test_labels = all_data[key][1]
        log_reg_score = log_reg.score(test_features, test_labels)
        rdm_for_score = rdm_for.score(test_features, test_labels)
        log_reg_f1score = f1_score(test_labels, log_reg.predict(test_features))
        rdm_for_f1score = f1_score(test_labels, rdm_for.predict(test_features))
        log_reg_auc = roc_auc_score(test_labels, log_reg.predict_proba(test_features)[:, 1])
        rdm_for_auc = roc_auc_score(test_labels, rdm_for.predict_proba(test_features)[:, 1])
        oos_accs.append([log_reg_auc, rdm_for_auc])
        
        print("Extrap to {0}, LR: {1}, RF: {2}".format(cont, round(log_reg_score, 5), round(rdm_for_score, 5)))
        print("F1 for LR: {0}, RF: {1}".format(round(log_reg_f1score, 4), round(rdm_for_f1score, 4)))
        print("AUC for LR: {0}, RF: {1}".format(round(log_reg_auc, 4), round(rdm_for_auc, 4)))
        print(np.unique(test_labels, return_counts=True))
#         print(np.unique(log_reg.predict(test_features), return_counts=True))
#         print(np.unique(rdm_for.predict(test_features), return_counts=True))
        print()

Extrap to eur, LR: 0.9695, RF: 0.99782
F1 for LR: 0.0, RF: 0.0
AUC for LR: 0.9432, RF: 0.4509
(array([0, 1]), array([458,   1]))

Extrap to asia, LR: 0.86788, RF: 0.98446
F1 for LR: 0.0377, RF: 0.0
AUC for LR: 0.6623, RF: 0.7596
(array([0, 1]), array([380,   6]))

Extrap to afr, LR: 0.95026, RF: 0.97599
F1 for LR: 0.0, RF: 0.0
AUC for LR: 0.4178, RF: 0.6913
(array([0, 1]), array([569,  14]))

Extrap to am, LR: 0.93119, RF: 0.98624
F1 for LR: 0.0625, RF: 0.0
AUC for LR: 0.5795, RF: 0.8798
(array([0, 1]), array([430,   6]))

Extrap to me, LR: 1.0, RF: 0.9907
F1 for LR: 1.0, RF: 0.6667
AUC for LR: 1.0, RF: 0.9994
(array([0, 1]), array([211,   4]))



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.

### Compare Accuracy Drops

In [76]:
i = 0
for key in all_data.keys():
    if 'test' in key:
        cont = key.split('_')[0]
        print("Drop in {0}: LR: {1}, RF: {2}".format(cont, round(baseline_accs[i][0] - oos_accs[i][0], 4), round(baseline_accs[i][1] - oos_accs[i][1], 4)))
        i += 1

Drop in eur: LR: 0.0109, RF: 0.5273
Drop in asia: LR: -0.0285, RF: -0.086
Drop in afr: LR: 0.0187, RF: 0.0122
Drop in am: LR: -0.0066, RF: -0.1128
Drop in me: LR: 0.0, RF: 0.0006
